### Monday, December 4, 2023

Trying it again ...

pip install exllamav2

docker container start hfpt_Oct28

### Sunday, December 3, 2023

Downloaded the "TheBloke/Python-Code-33B-GPTQ" model yesterday ... ran a quick test this morning. It works!

### Friday, November 17, 2023

https://huggingface.co/TheBloke/Python-Code-33B-GPTQ

In [1]:
# You need this to just target the 4090.
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
model_name_or_path = "TheBloke/Python-Code-33B-GPTQ"

In [3]:
!ls /home/rob/Data2/huggingface/transformers

models--TheBloke--CodeLlama-34B-Instruct-GPTQ  tmpcjh0h7gn
models--TheBloke--Llama-2-13b-Chat-GPTQ        tmpzafytbf_
models--TheBloke--Python-Code-33B-GPTQ	       version.txt
models--meta-llama--Llama-2-13b-hf


In [3]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

In [4]:
# This loads the model into CPU ram for inference 
# hf = HuggingFacePipeline.from_model_id(
#     model_id=model_name_or_path,
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 512},
# )

# This too loads it to CPU ram 
# hf = HuggingFacePipeline.from_model_id(
#     model_id=model_name_or_path,
#     task="text-generation",
#     device=0,
#     pipeline_kwargs={"max_new_tokens": 512},
# )

# This too loads it to CPU ram 
# hf = HuggingFacePipeline.from_model_id(
#     model_id=model_name_or_path,
#     task="text-generation",
#     device_map="auto",
#     pipeline_kwargs={"max_new_tokens": 512},
# )

# This takes up all CPU ram, then fill up ALL swap space ... yeah, this does not work!
# Also, I stopped it at 6 minutes cuz it was gonna fill up the swap space ... and that never ends well.
# I also tried this without running CUDA_VISIBLE_DEVICES ... same problem.
hf = HuggingFacePipeline.from_model_id(
    model_id=model_name_or_path,
    task="text-generation",
    device_map="auto",
    trust_remote_code=False,
    pipeline_kwargs={"max_new_tokens": 512},
)

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


KeyboardInterrupt: 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [6]:
%%time
# To use a different branch, change revision
# For example: revision="gptq-4bit-128g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             # use_exllama_v2=True, ... this does not work
                                             revision="main")


# Load time
# CPU times: user 19.9 s, sys: 9.32 s, total: 29.3 s
# Wall time: 12.4 s
# 18424MiB VRAM
# Memory goes from 4.9GB to 10.8GB ... why??

# Download time
# CPU times: user 1min 32s, sys: 1min 39s, total: 3min 12s
# Wall time: 4h 37s

You have activated exllama backend. Note that you can get better inference
                    speed using exllamav2 kernel by setting `use_exllama_v2=True`.`disable_exllama` will be deprecated
                    in future version.


CPU times: user 20.7 s, sys: 11.2 s, total: 31.9 s
Wall time: 14.6 s


In [7]:
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--TheBloke--Python-Code-33B-GPTQ /home/rob/Data3/huggingface/transformers
# Successfully copied 16.9GB to /home/rob/Data3/huggingface/transformers

In [7]:
prompt = "Tell me about AI"
prompt_template=f'''This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: {prompt}
ASSISTANT:
'''

# print("\n\n*** Generate:")

In [8]:
input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

In [9]:
%%time
maxTokens = 2048
output2048 = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=maxTokens)
output = output2048[0]
print(len(output))
print(tokenizer.decode(output))

# This cell starts at 18424 MiB VRAM ... goes up to 19524 MiB
# 9.9s ... WHY does this time vary by so much?? This was fast!
# It varies based on how many tokens get generated for the output ...
# 708 ... 32.9s ...
# 371 ... 16.1s ...
# 595 ... 26.1s ...
# 405 ... 17.0s ...
# 632 ... 28.4s ...


# 38.4s
# 20.8s

632
<s> This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:

Artificial Intelligence (AI) is a subfield of computer science that studies intelligent agents. An intelligent agent is a program that can make decisions and take actions based on its environment. 

AI researchers study how to make machines intelligent by replicating the processes of the brain. They use techniques such as machine learning, pattern recognition, and natural language processing to achieve this. 

AI is a vast field with many applications. Some common applications include:

1. Machine Learning: This subfield of AI studies how machines can learn from data and improve their performance. It is used in many practical applications such as spam filtering, recommendation systems, and self-driving cars.

2. Pattern Recognition: This subfield studies how machines can recogn

In [10]:
type(output2048)

torch.Tensor

In [11]:
# Inference can also be done using transformers' pipeline

# print("*** Pipeline:")
tfPipeLine = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

In [12]:
print(tfPipeLine(prompt_template)[0]['generated_text'])

# 21106 MiB VRAM
# 20324 MiB VRAM
# 19906 MiB VRAM

# 18.7s
# 15.0s
# 12.9s 
# 15.2s
# 10.9s

This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: Tell me about AI
ASSISTANT:
Artificial Intelligence (AI) is a field of computer science that studies intelligent agents. An intelligent agent is a program that can make decisions and take actions on its own, without human intervention. 

AI assistants, such as Siri, Alexa, and Google Assistant, have become common in recent years. These assistants use natural language processing (NLP) to understand human speech and respond accordingly. They can perform tasks like searching the internet, playing music, and controlling other devices in the home.

In addition to consumer-level AI assistants, there are also advanced AI systems being developed for use in industries like finance, healthcare, and transportation. These systems use machine learning algorithms to analyze large amounts of data and make predictions about futur

So let's try this with [LangChain](https://python.langchain.com/docs/integrations/llms/huggingface_pipelines), shall we ... 

In [13]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline as LCHuggingFacePipeLine

In [14]:
# pass in the transformers pipeline we defined above ... 
lchfpl = LCHuggingFacePipeLine(pipeline=tfPipeLine, verbose=True )

Below is the replicated code from a previous cell ... 

In [21]:
prompt = "Tell me about AI"

prompt_template=f'''This is a conversation with your helpful AI assistant. AI assistant can generate Python Code along with necessary explanation.

Context
You are a helpful AI assistant.

USER: {prompt}
ASSISTANT:
'''


In [15]:
from langchain.prompts import PromptTemplate as LCPromptTemplate

lcPromptTemplate = LCPromptTemplate.from_template(prompt_template)

chain = lcPromptTemplate | lchfpl

In [16]:

result = chain.invoke({"prompt": prompt})

In [17]:
print(result)

Artificial Intelligence (AI) is a subfield of computer science that studies intelligent agents. An intelligent agent is a system that perceives its environment and takes actions to achieve specific goals. 

In the past few decades, AI has made significant progress in various fields such as speech recognition, image processing, and machine learning. These advances have enabled AI systems to perform tasks that were previously considered to be the domain of humans, such as playing chess or identifying faces.

However, there are still many challenges to overcome before AI systems can match the full range of human capabilities. One such challenge is understanding language, which is essential for interacting with humans. Another challenge is improving the efficiency of the algorithms used by AI systems, so that they can handle larger amounts of data and make more accurate predictions.

Overall, the field of AI is growing rapidly, and it is expected to have a significant impact on many aspect